In [2]:
import cv2
import scipy.io
import numpy as np
import skimage
from scipy import signal
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [13]:
def pyr_reduce(img):
    kernel_width = 5  # default
    cw = 0.375  # kernel center weight, same as MATLAB func impyramid. 0.6 in the Paper
    ker1d = np.array([0.25 - cw / 2, 0.25, cw, 0.25, 0.25 - cw / 2])
    kernel = np.outer(ker1d, ker1d)

    step = 2
    img = skimage.img_as_float(img)
    reduced_shape = np.ceil(np.array(img.shape) / 2).astype(int)
    img_out = np.empty(reduced_shape)

    if(img.ndim > 2): # For RGB images
        for p in range(img.shape[2]):
            img1 = img[:, :, p]
            img_filtered = signal.convolve2d(img1, kernel, mode='same', boundary='fill', fillvalue=0)
            img_out[:, :] = img_filtered[::step, ::step]

    else: # For grayscale images
        img1 = img[:, :]
        img_filtered = signal.convolve2d(img1, kernel, mode='same', boundary='fill', fillvalue=0)
        img_out[:, :] = img_filtered[::step, ::step]


    return img_out


def pyr_expand(img):
    kw = 5
    cw = 0.375
    ker1d = [0.25 - cw/2, 0.25, cw, 0.25, 0.25 - cw/2]
    kernel = np.outer(ker1d, ker1d) * 4

    ker00 = kernel[:kw:2, :kw:2]
    ker01 = kernel[:kw:2, 1:kw:2]
    ker10 = kernel[1:kw:2, :kw:2]
    ker11 = kernel[1:kw:2, 1:kw:2]

    img = img.astype(np.float64)
    sz = img.shape[:2]
    osz = (sz[0] * 2 - 1, sz[1] * 2 - 1)

    if(img.ndim > 2): # For RGB images
        imgout = np.zeros((osz[0], osz[1], img.shape[2]))
        for p in range(img.shape[2]):
            img1 = img[:, :, p]
            img1ph = np.pad(img1, [(0,0), (1,1)], mode='edge')
            img1pv = np.pad(img1, [(1,1), (0,0)], mode='edge')

            img00 = scipy.signal.convolve2d(img1, ker00, mode='same', boundary='fill', fillvalue=0)
            img01 = scipy.signal.convolve2d(img1pv, ker01, mode='valid')
            img10 = scipy.signal.convolve2d(img1ph, ker10, mode='valid')
            img11 = scipy.signal.convolve2d(img1, ker11, mode='valid')

            imgout[::2, ::2, p] = img00
            imgout[1::2, ::2, p] = img10
            imgout[::2, 1::2, p] = img01
            imgout[1::2, 1::2, p] = img11

    else: # For grayscale images
        imgout = np.zeros((osz[0], osz[1]))
        img1 = img[:, :]
        img1ph = np.pad(img1, [(0,0), (1,1)], mode='edge')
        img1pv = np.pad(img1, [(1,1), (0,0)], mode='edge')

        img00 = scipy.signal.convolve2d(img1, ker00, mode='same', boundary='fill', fillvalue=0)
        img01 = scipy.signal.convolve2d(img1pv, ker01, mode='valid')
        img10 = scipy.signal.convolve2d(img1ph, ker10, mode='valid')
        img11 = scipy.signal.convolve2d(img1, ker11, mode='valid')

        imgout[::2, ::2] = img00
        imgout[1::2, ::2] = img10
        imgout[::2, 1::2] = img01
        imgout[1::2, 1::2] = img11

    return imgout


def pyrReconstruct(pyr):
    for p in range(len(pyr)-1, 0, -1):
        pyr[p-1] = pyr[p-1] + pyr_expand(pyr[p])
    img = pyr[0]
    return img


def genPyr(img, type, level):
    # Create a list of zeros with the specified level
    pyr = [np.zeros((img['Z'].shape[0], img['Z'].shape[1]), dtype=np.double) for i in range(level)]
    pyr[0] = img['Z'].astype(np.double)

    for p in range(1, level):
        # Reduce i-1'th scale
        pyr[p] = pyr_reduce(pyr[p-1])

    if type == 'gauss':
        return pyr

    # When expanding, we may get image dimensions that
    # aren't quite the same when targetting a particular scale
    # As such, we need to make sure we are consistent
    for p in range(level-2, -1, -1):
        osz = (pyr[p+1].shape[0] * 2 - 1, pyr[p+1].shape[1] * 2 - 1)
        pyr[p] = pyr[p][:osz[0], :osz[1]]

    for p in range(0, level-1):
        # Take i'th scale and subtract with expanded i+1'th scale
        pyr[p] = pyr[p] - pyr_expand(pyr[p+1])

    return pyr


def gsr(S, pyr, levels, scales):
    for i in range(levels):
        if i in scales:
            pyr[i] = np.zeros(pyr[i].shape)

    D = pyr[0].shape
    Sp = {
        'Z': pyrReconstruct(pyr),
        'X': S['X'][:D[0],:D[1]],
        'Y': S['Y'][:D[0],:D[1]],
    }
    return Sp

In [ ]:
def process_3D_images(input_path, output_path, n):
  # Function for obtaining the 3D representation of a surface in
  # n different size versions

  # Adapted version to translate to Python language using ChatGPT3
  # http://opilab.utb.edu.co/
  # This code is part of research work done by Jesus Pineda Castro.
  # Universidad Tecnológica de Bolívar , Cartagena, Colombia.

  # Load data
  data = scipy.io.loadmat(input_path)
  S = {}
  S['Z'] = data['ZZ'] * data['Indix']
  S['X'] = data['XX'] * data['Indix']
  S['Y'] = data['YY'] * data['Indix']

  # Get ROI
  plt.imshow(S['Z'])
  plt.axis('off')
  plt.colorbar()
  #plt.show()

  points = plt.ginput(2)
  x1, y1 = int(points[0][0]), int(points[0][1])
  x2, y2 = int(points[1][0]), int(points[1][1])

  # Crop the image
  S['Z'] = S['Z'][y1:y2, x1:x2]
  S['X'] = S['X'][y1:y2, x1:x2]
  S['Y'] = S['Y'][y1:y2, x1:x2]

  # THIS IS A PROOF BY ME
  surface = np.stack((S['X'], S['Y'], S['Z']), axis=2)

  # Global surface removal
  levels = 7
  #pyr = genPyr(S, 'lap', levels)
  pyr = genPyr(surface, 'lap', levels)
  Sp = gsr(S, pyr, levels, [1, 2, 7])
  #Sp = gsr(surface, pyr, levels, [1, 2, 7])

  # Display results
  l = [30, 70]  # mask limits
  mask = np.empty(Sp['Z'].shape) * np.nan
  mask[l[0]:-l[1], l[0]:-l[0]] = 1
  Spz = Sp['Z'] * mask
  Spz[np.isnan(Spz)] = 0

  # Gaussian Pyramid
  g_levels = n
  gauss_pyr = genPyr(Sp, 'gauss', g_levels)

  # Display pyramid levels
  Spz_pyr = [None] * g_levels
  for i in range(g_levels):
      mask2 = np.empty(gauss_pyr[i].shape) * np.nan
      mask2[l[0]:-l[1], l[0]:-l[0]] = 1
      Spz_pyr[i] = gauss_pyr[i] * mask2
      Spz_pyr[i][np.isnan(Spz_pyr[i])] = 0
      l = np.ceil(l / 2).astype(int)

  # Saving results
  np.save(output_path, Spz_pyr)

In [14]:
def process_3D_images(input_path, output_path):
  # Function for obtaining the 3D representation of a surface in
  # n different size versions

  # Adapted version to translate to Python language using ChatGPT3
  # http://opilab.utb.edu.co/
  # This code is part of research work done by Jesus Pineda Castro.
  # Universidad Tecnológica de Bolívar , Cartagena, Colombia.

  # Load data
  data = scipy.io.loadmat(input_path)
  S = {}
  S['Z'] = data['ZZ'] * data['Indix']
  S['X'] = data['XX'] * data['Indix']
  S['Y'] = data['YY'] * data['Indix']

  # Get ROI
  plt.imshow(S['Z'])
  plt.axis('off')
  plt.colorbar()
  #plt.show()

  points = plt.ginput(2)
  x1, y1 = int(points[0][0]), int(points[0][1])
  x2, y2 = int(points[1][0]), int(points[1][1])

  # Crop the image
  S['Z'] = S['Z'][y1:y2, x1:x2]
  S['X'] = S['X'][y1:y2, x1:x2]
  S['Y'] = S['Y'][y1:y2, x1:x2]

  # Global surface removal
  levels = 7
  pyr = genPyr(S, 'lap', levels)
  #pyr = genPyr(surface, 'lap', levels)
  Sp = gsr(S, pyr, levels, [1, 2, 7])
  #Sp = gsr(surface, pyr, levels, [1, 2, 7])

  # Display results
  l = [30, 70]  # mask limits
  mask = np.empty(Sp['Z'].shape) * np.nan
  mask[l[0]:-l[1], l[0]:-l[0]] = 1
  Spz = Sp['Z'] * mask
  Spz[np.isnan(Spz)] = 0

  # Saving results
  np.save(output_path, Spz)

In [15]:
num = '2'
input_path = f'../Data/MATLAB/input/data{num}_R3D.mat'
output_path = f'../Data/MATLAB/output/data{num}.npy'
#n = 3 # Number of scaled images

In [16]:
#process_3D_images(input_path, output_path, n)
process_3D_images(input_path, output_path)

In [24]:
data = np.load(f'../Data/MATLAB/output/data{num}.npy')
plt.imshow(data)
plt.show()